In [1]:
%%capture
import numpy as np
import pandas as pd
!pip install openpyxl
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import os

In [2]:
actual_path = os.getcwd()
os.chdir('/home/onyxia/work/material_science/Spetral_clustering')
%run rbf.ipynb
os.chdir(actual_path)

In [3]:
max_calculated = pd.read_csv('max_calculated.csv')
max_elemental = pd.read_csv('max_elemental.csv')
list_mxene = pd.read_excel('synthesized-MXenes-MAX.xlsx',sheet_name=0)
list_failed = pd.read_excel('synthesized-MXenes-MAX.xlsx', sheet_name=2)
n_samples = max_elemental.shape[0]
synth_list = pd.unique(list_mxene['MXene'])[:-1]
to_drop = list(range(167,173))
mx_ene_df = list_mxene.drop(labels = to_drop, axis='index')
mx_ene_df = mx_ene_df.drop(['Unnamed: 9','Unnamed: 12','Notes','status','Reference method'],axis=1)
max_elemental['class'] = np.zeros(max_elemental.shape[0])
parents = mx_ene_df['Parent material'].unique()
banned_words = ['+','Mxene','topochemical','reaction', 'or',
               'synthesis','MXene','direct']
complete_parents = []
for i in range(len(parents)):
    inter = parents[i].split()
    for word in range(len(inter)):
        if inter[word] not in banned_words:
            complete_parents.append(inter[word])


for i in range(max_elemental.shape[0]):
    if max_elemental.loc[i,'compound_formula'] in complete_parents:
        max_elemental.loc[i,'class'] = 1

max_elemental = max_elemental.set_index('compound_formula',drop=True)
max_elemental = max_elemental.drop(['M_element', 'A_element', 'X_element'],axis=1)
max_calculated = max_calculated.set_index('prettyformula',drop=True)
whole_data = max_elemental.merge(max_calculated,how='inner',
                                 left_index=True,right_index=True)
whole_data = whole_data.drop(['PU_label','year'],axis=1)
M_elements = pd.get_dummies(whole_data['M'],prefix='M',dtype=float)
A_elements = pd.get_dummies(whole_data['A'],prefix='A',dtype=float)
X_elements = pd.get_dummies(whole_data['X'],prefix='X',dtype=float)
whole_data = whole_data.drop(['M','A','X'],axis=1)
x_group = pd.get_dummies(whole_data['X_X_group'],prefix='x_g',dtype=float)
a_group = pd.get_dummies(whole_data['A_A_group'],prefix='a_g',dtype=float)
m_group = pd.get_dummies(whole_data['M_M_group'],prefix='m_g',dtype=float)
whole_data = whole_data.drop(['X_X_group','A_A_group','M_M_group'],axis=1)
whole_data = pd.concat([whole_data,M_elements,A_elements,X_elements,x_group,
                       a_group,m_group],axis=1)

test_tree = DecisionTreeClassifier().fit(X=whole_data.drop(['class'],axis=1),
                                                              y=whole_data['class'])

imp_feat = test_tree.feature_importances_
names_feat = test_tree.feature_names_in_

imp_feat = imp_feat.reshape(-1,1)
names_feat = names_feat.reshape(-1,1)
test_df = pd.DataFrame(np.hstack((names_feat,imp_feat)))
test_df.columns = ['names_feat','imp_feat']
test_df = test_df.set_index('names_feat',drop=True)
test_df = test_df[test_df['imp_feat'] > 0]

diff_z = list(test_df.index)


number_of_atoms = np.zeros(n_samples)
compteur = 0
for element in whole_data.index:
    inter = []
    for cara in element:
        if cara in list(str(1234567890)):
            inter.append(cara)
    if len(inter) == 1:
        number_of_atoms[compteur] = int(inter[0]) + 2
    elif len(inter) == 2:
        number_of_atoms[compteur] = int(inter[0]) + int(inter[1]) + 1
    elif len(inter) == 3:
        number_of_atoms[compteur] = int(inter[0]) + int(inter[1]) + int(inter[2])
    compteur += 1

columns_name = whole_data.drop(['class'],axis=1).columns.copy()
normalized = whole_data.drop(['class'],axis=1).to_numpy()/number_of_atoms.reshape(-1,1)

data_norm = pd.DataFrame(normalized)
data_norm.columns = columns_name
data_norm['compound_name'] = whole_data.index
data_norm = data_norm.set_index('compound_name',drop=True)

data_norm = data_norm.filter(items=list(diff_z),axis=1)
data_norm['class'] = whole_data['class'].copy()

retained_features = list(test_df.index)

for feat in diff_z:
    if len(feat) > 5:
        retained_features.remove(feat)

list_dummies = []

for i in retained_features:
    if 'M_' in i:
        list_dummies.append(i)
    elif 'A_' in i:
        list_dummies.append(i)
    elif 'X_' in i:
        list_dummies.append(i)

for col in list_dummies:
    for row in data_norm.index:
        if data_norm.loc[row,col] != 0:
            data_norm.loc[row,col] = 1

for i in data_norm.index:
    if data_norm.loc[i,'class'] == 0:
        data_norm.loc[i,'class'] = -1

In [4]:
very_positive_instances = data_norm[data_norm['class'] == 1].copy()

In [5]:
very_positive_instances['pred'] = np.zeros(15)

In [6]:
list_of_positive_part = list(very_positive_instances.index)

In [7]:
list_wo_class = list(data_norm.columns)
list_wo_class.remove('class')
df_whole_results = data_norm.drop(list_wo_class,axis=1)
t=15
count_elem = 0
n_positives = very_positive_instances.shape[0]-1
for part_element in list_of_positive_part:
    data = data_norm.drop(part_element,axis=0).copy()
    df_results = data.drop(list(data.columns),axis=1)
    positive_instances = data[data['class'] == 1]
    index_of_positive = list(positive_instances.index)
    unlabelled_instances = data.drop(labels=index_of_positive,axis=0)
    avg_element = np.zeros(t)
    for i in range(t):
        u_t = unlabelled_instances.sample(n=n_positives)
        training_set = pd.concat([positive_instances,u_t])
        training_index = list(training_set.index)
        clf = SVC().fit(X=training_set.drop(['class'],axis=1).to_numpy(),y=training_set['class'].to_numpy())
        interm_df = data.drop(labels=training_index,axis=0)
        interm_df['predictions'] = clf.decision_function(X=interm_df.drop(['class'],axis=1).to_numpy())
        df_for_merge = pd.DataFrame(interm_df['predictions'])
        df_for_merge.columns = [f'score_{i},{count_elem}']
        df_results = df_results.merge(df_for_merge, how='left', left_index=True, right_index=True)
        avg_element[i] = clf.decision_function(X=data_norm.drop(['class'],axis=1).loc[part_element,:].to_numpy().reshape(1,-1))[0]
    very_positive_instances.loc[part_element,'pred'] = np.mean(avg_element)
    df_whole_results = df_whole_results.merge(df_results,how='left',left_index=True,right_index=True)
    count_elem += 1

In [8]:
avg_results = df_whole_results.drop(['class'],axis=1).mean(axis=1)

In [9]:
very_positive_instances

,M_M_van_der_waals_radius,A_A_second_ionization_energy,c,dMX,dMA,rMA,e_A,e_X,dEc,density,dEf,dH,M_Ti,M_V,A_Al,A_Zn,class,pred
compound_name,,,,,,,,,,,,,,,,,,
Ti2AlC,0.527500,4.707137,3.434875,0.286710,0.572010,0.723328,-0.177815,-0.481092,1585.313992,0.999209,-175.367182,-4.433290,1.0,0.0,1.0,0.0,1.0,0.573128
Ti2SC,0.527500,5.834470,2.817900,0.279593,0.424882,0.627973,-0.298357,-0.468533,1705.123542,1.158582,-312.925067,-14.156324,1.0,0.0,0.0,0.0,1.0,0.793050
Ti2ZnC,0.527500,4.491098,3.441525,0.288608,0.571775,0.722298,-0.187100,-0.483255,1396.775752,1.289542,-135.340198,6.033368,1.0,0.0,0.0,1.0,1.0,0.353487
V2AlC,0.517500,4.707137,3.282425,0.279170,0.541435,0.685475,-0.063488,-0.411033,1507.684761,1.212639,-128.911353,-15.625497,0.0,1.0,1.0,0.0,1.0,0.478091
Nb2AlC,0.545000,4.707137,3.485475,0.311915,0.559453,0.719043,-0.055508,-0.426833,1702.981879,1.578442,-126.534736,-4.223056,0.0,0.0,1.0,0.0,1.0,0.486352
Ta2AlC,0.555000,4.707137,3.491000,0.314817,0.557933,0.714878,-0.015805,-0.456950,1884.426031,2.870454,-131.844990,-2.501170,0.0,0.0,1.0,0.0,1.0,0.383437
Ti3AlC2,0.351667,3.138092,3.109183,0.197525,0.382243,0.483758,-0.113453,-0.300678,1128.862198,0.702228,-127.286053,-1.419006,1.0,0.0,1.0,0.0,1.0,-0.015598
Ti3SiC2,0.351667,2.724308,2.955250,0.199508,0.339795,0.450538,-0.154085,-0.296233,1168.330449,0.746292,-137.080337,-1.351975,1.0,0.0,0.0,0.0,1.0,-0.011209
Ti3ZnC2,0.351667,2.994065,3.103967,0.197908,0.380173,0.481942,-0.123367,-0.302950,1046.059042,0.843894,-110.487900,-1.186989,1.0,0.0,0.0,1.0,1.0,-0.317415


In [10]:
very_positive_instances['pred_bis'] = np.sign(very_positive_instances['pred'])

In [11]:
for_recall = very_positive_instances['pred_bis'].to_numpy()

In [12]:
true_positives = 0
for i in range(15):
    if for_recall[i] == 1:
        true_positives += 1

In [13]:
recall = true_positives/15

In [14]:
df_whole_results = df_whole_results.drop(['class'],axis=1)

In [15]:
df_avg = df_whole_results.mean(axis=1)

In [16]:
df_avg = pd.DataFrame(df_avg)
df_avg.columns = ['avg']
df_avg['decision'] = np.sign(df_avg['avg'])

In [17]:
predicted_positives = df_avg.groupby('decision').count().loc[1.,'avg']

In [18]:
proba_pos_pred = predicted_positives/data_norm.shape[0]

In [19]:
weird_metric = (recall**2)/proba_pos_pred

In [20]:
weird_metric

np.float64(0.5920833333333334)

In [21]:
recall

0.4666666666666667

In [25]:
df_whole_np = df_whole_results.to_numpy()

In [38]:
avg_df = np.zeros((2262,15))
for i in range(15):
    avg_df[:,i] = np.nanmean(df_whole_np[:,14*i:14*(i+1)],axis=1)

/tmp/ipykernel_22178/2250680610.py:3: RuntimeWarning: Mean of empty slice
  avg_df[:,i] = np.nanmean(df_whole_np[:,14*i:14*(i+1)],axis=1)


In [43]:
avg_df = (np.sign(avg_df)+1)/2

In [46]:
avg_proba = np.nanmean(avg_df)

In [47]:
avg_proba

np.float64(0.37709538644118085)

In [48]:
weird_metric = recall/avg_proba

In [49]:
weird_metric

np.float64(1.2375295043273014)